|        |        |        |
|--------|--------|--------|
![H-BRS](logos/h-brs.png) | ![A2S](logos/a2s.png) | ![b-it](logos/b-it.png) |

# Software Engineering for Robotics

# SER Assignment 1

#### Team members:
* ememis2s
* ...
* ...

# Robot Behaviour Management Using State Machines and Behaviour Trees

In this assignment, we will compare the implementation of behaviour trees and state machines to establish primary safety features for a robot; this includes situations such as the battery level falling below a certain threshold and avoiding potential collisions. In other words, we want to implement both a behaviour tree and a state machine to achieve the same functionality.

We particularly want the robot to behave as follows:
* *The battery level falling below a given threshold*: The robot starts rotating in place until the level is above the threshold again (**you can control the battery level by publishing to a topic you define**).
* *A collision is about to happen*: The robot stops moving and needs to be moved to a safe distance manually.

**You will use the Robile simulation for testing your implementation. For your submission, you need to add your code to the appropriate cells below; however, note that, to actually test your implementation, you need to integrate the code in a ROS package and perform all tests on your local machine.**

## Robot Safety Functionalities Using a State Machine [45 points]

To implement a state machine, we will use the [SMACH framework](https://wiki.ros.org/smach/Tutorials). SMACH as such is ROS independent, but `executive_smach` provides a ROS integration, which we will be using in this assignment.

Your task here is to implement the necessary states and set up the state machine to achieve the desired robot functionality.

In [2]:
"""
Robot Safety State Machine using SMACH
Implements battery monitoring and collision avoidance
"""

import threading
import time
from dataclasses import dataclass

import rclpy
import smach
from geometry_msgs.msg import Twist
from rclpy.node import Node
from sensor_msgs.msg import LaserScan
from std_msgs.msg import Float32


@dataclass
class SafetyConfig:
    """Centralized configuration - Single source of truth!"""

    # Battery thresholds
    low_battery_threshold: float = 20.0
    battery_ok_threshold: float = 85.0

    # Collision thresholds
    collision_distance: float = 0.5
    safe_distance: float = 0.75

    # Robot motion
    rotation_speed: float = 5.0
    monitor_rate: float = 10.0

    @classmethod
    def from_node(cls, node: Node):
        """Load configuration from ROS2 parameters"""
        # Declare all parameters with defaults from this class
        node.declare_parameter("low_battery_threshold", cls.low_battery_threshold)
        node.declare_parameter("battery_ok_threshold", cls.battery_ok_threshold)
        node.declare_parameter("collision_distance", cls.collision_distance)
        node.declare_parameter("safe_distance", cls.safe_distance)
        node.declare_parameter("rotation_speed", cls.rotation_speed)
        node.declare_parameter("monitor_rate", cls.monitor_rate)

        # Read and return config
        return cls(
            low_battery_threshold=node.get_parameter("low_battery_threshold").value,
            battery_ok_threshold=node.get_parameter("battery_ok_threshold").value,
            collision_distance=node.get_parameter("collision_distance").value,
            safe_distance=node.get_parameter("safe_distance").value,
            rotation_speed=node.get_parameter("rotation_speed").value,
            monitor_rate=node.get_parameter("monitor_rate").value,
        )

    def log_config(self, logger):
        """Pretty print configuration"""
        logger.info("=" * 60)
        logger.info("🔧 SMACH Safety Configuration:")
        logger.info(f"  Battery Low:      {self.low_battery_threshold:.1f}%")
        logger.info(f"  Battery OK:       {self.battery_ok_threshold:.1f}%")
        logger.info(f"  Collision Dist:   {self.collision_distance:.2f}m")
        logger.info(f"  Safe Dist:        {self.safe_distance:.2f}m")
        logger.info(f"  Rotation Speed:   {self.rotation_speed:.1f} rad/s")
        logger.info("=" * 60)


class MonitorBatteryAndCollision(smach.State):
    """State to monitor battery level and collisions"""

    def __init__(self, node: Node, config: SafetyConfig):
        smach.State.__init__(self, outcomes=["battery_low", "collision", "all_good"])
        self.node = node
        self.config = config

        # State variables
        self.battery_level = 100.0
        self.min_scan_distance = float("inf")

        # Subscribers
        self.battery_sub = node.create_subscription(Float32, "/battery_level", self.battery_callback, 10)
        self.scan_sub = node.create_subscription(LaserScan, "/scan", self.scan_callback, 10)

        node.get_logger().info("✅ MonitorBatteryAndCollision state initialized")

    def battery_callback(self, msg):
        self.battery_level = msg.data

    def scan_callback(self, msg):
        valid_ranges = [r for r in msg.ranges if 0.1 < r < msg.range_max]
        self.min_scan_distance = min(valid_ranges) if valid_ranges else float("inf")

    def execute(self, userdata):
        self.node.get_logger().info(f"👁️  Monitoring - Battery: {self.battery_level:.1f}%, Min dist: {self.min_scan_distance:.2f}m")

        # Check collision first (higher priority)
        if self.min_scan_distance < self.config.collision_distance:
            self.node.get_logger().warn(f"⚠️  COLLISION! Obstacle at {self.min_scan_distance:.2f}m")
            return "collision"

        # Check battery
        if self.battery_level < self.config.low_battery_threshold:
            self.node.get_logger().warn(f"🔋 LOW BATTERY! Level: {self.battery_level:.1f}%")
            return "battery_low"

        time.sleep(0.1)
        return "all_good"


class RotateBase(smach.State):
    """State to rotate robot for recharging"""

    def __init__(self, node: Node, config: SafetyConfig):
        smach.State.__init__(self, outcomes=["battery_ok"])
        self.node = node
        self.config = config

        # State variables
        self.battery_level = 0.0

        # Subscriber
        self.battery_sub = node.create_subscription(Float32, "/battery_level", self.battery_callback, 10)

        # Publisher
        self.cmd_vel_pub = node.create_publisher(Twist, "/cmd_vel", 10)

        node.get_logger().info("✅ RotateBase state initialized")

    def battery_callback(self, msg):
        self.battery_level = msg.data

    def execute(self, userdata):
        self.node.get_logger().info(f"🔄 ROTATING to recharge - Battery: {self.battery_level:.1f}%")

        # Keep rotating until recharged
        rate = self.node.create_rate(10)
        while self.battery_level < self.config.battery_ok_threshold:
            # Publish rotation command
            twist = Twist()
            twist.angular.z = self.config.rotation_speed
            self.cmd_vel_pub.publish(twist)

            try:
                rate.sleep()
            except Exception:
                pass

        # Stop rotating
        twist = Twist()
        twist.angular.z = 0.0
        self.cmd_vel_pub.publish(twist)

        self.node.get_logger().info(f"✅ Battery recharged to {self.battery_level:.1f}%")

        return "battery_ok"


class StopMotion(smach.State):
    """State to stop robot when collision detected"""

    def __init__(self, node: Node, config: SafetyConfig):
        smach.State.__init__(self, outcomes=["safe_distance"])
        self.node = node
        self.config = config

        # State variables
        self.min_scan_distance = 0.0

        # Subscriber
        self.scan_sub = node.create_subscription(LaserScan, "/scan", self.scan_callback, 10)

        # Publisher
        self.cmd_vel_pub = node.create_publisher(Twist, "/cmd_vel", 10)

        node.get_logger().info("✅ StopMotion state initialized")

    def scan_callback(self, msg):
        valid_ranges = [r for r in msg.ranges if 0.1 < r < msg.range_max]
        self.min_scan_distance = min(valid_ranges) if valid_ranges else float("inf")

    def execute(self, userdata):
        self.node.get_logger().info(f"🛑 STOPPED - Obstacle at {self.min_scan_distance:.2f}m")

        # Keep stopping until obstacle cleared
        rate = self.node.create_rate(10)
        while self.min_scan_distance <= self.config.safe_distance:
            # Publish stop command
            twist = Twist()
            twist.linear.x = 0.0
            twist.angular.z = 0.0
            self.cmd_vel_pub.publish(twist)

            try:
                rate.sleep()
            except Exception:
                pass

        self.node.get_logger().info(f"✅ Obstacle cleared! Distance: {self.min_scan_distance:.2f}m")

        return "safe_distance"


def main(args=None):
    """Main function to initialize and execute the state machine"""

    rclpy.init(args=args)

    # Create ROS2 node
    node = Node("safety_smach_node")

    # Load configuration
    config = SafetyConfig.from_node(node)
    config.log_config(node.get_logger())

    node.get_logger().info("🏗️  Building state machine...")

    # Create state machine
    sm = smach.StateMachine(outcomes=["shutdown"])

    # Add states
    with sm:
        smach.StateMachine.add("MONITOR_BATTERY_AND_COLLISION", MonitorBatteryAndCollision(node, config), transitions={"battery_low": "ROTATE_BASE", "collision": "STOP_MOTION", "all_good": "MONITOR_BATTERY_AND_COLLISION"})

        smach.StateMachine.add("ROTATE_BASE", RotateBase(node, config), transitions={"battery_ok": "MONITOR_BATTERY_AND_COLLISION"})

        smach.StateMachine.add("STOP_MOTION", StopMotion(node, config), transitions={"safe_distance": "MONITOR_BATTERY_AND_COLLISION"})

    node.get_logger().info("✅ State machine configured")
    node.get_logger().info("🚀 Starting execution...")

    # Execute state machine in separate thread
    sm_thread = threading.Thread(target=sm.execute)
    sm_thread.start()

    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        node.get_logger().info("⏹️  Shutting down...")
    finally:
        # Stop the robot
        cmd_vel_pub = node.create_publisher(Twist, "/cmd_vel", 10)
        stop_twist = Twist()
        cmd_vel_pub.publish(stop_twist)

        node.destroy_node()
        rclpy.shutdown()
        sm_thread.join()


if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'rclpy'

## Robot Safety Functionalities Using a Behaviour Tree [45 points]

The majority of implementations of behaviour trees in robotics are using `BehaviorTree.CPP` in cpp and [py_trees](https://py-trees.readthedocs.io/en/devel/) in Python. [py_trees_ros](https://py-trees-ros-tutorials.readthedocs.io/en/release-2.1.x/tutorials.html) is a wrapper for `py_trees` to integrate it with ROS, which we will use in this assignment.

Your task here is to implement the necessary behaviours and set up the behaviour tree to achieve the desired robot functionality.

Implement the required behaviours in the cell below. [35 points]

In [3]:
"""
Behavior Tree Behaviors for Robot Safety System
"""

import py_trees as pt
import py_trees_ros as ptr
from geometry_msgs.msg import Twist
from rclpy.qos import QoSHistoryPolicy, QoSProfile, QoSReliabilityPolicy
from sensor_msgs.msg import LaserScan
from std_msgs.msg import Float32


class Rotate(pt.behaviour.Behaviour):
    """Rotates the robot about the z-axis for battery recharging"""

    def __init__(self, name="Rotate", topic_name="/cmd_vel", ang_vel=5.0):
        super(Rotate, self).__init__(name)
        self.topic_name = topic_name
        self.ang_vel = ang_vel
        self.cmd_vel_pub = None
        self.blackboard = None

    def setup(self, **kwargs):
        """Setup publisher and blackboard client"""
        try:
            self.node = kwargs["node"]
        except KeyError as e:
            error_message = f"didn't find 'node' in setup's kwargs [{self.qualified_name}]"
            raise KeyError(error_message) from e

        # Create publisher
        self.cmd_vel_pub = self.node.create_publisher(Twist, self.topic_name, 10)

        # Attach to blackboard
        self.blackboard = self.attach_blackboard_client(name=self.name)
        self.blackboard.register_key(key="battery_low_warning", access=pt.common.Access.READ)

        self.node.get_logger().info(f"[{self.name}] ✅ Setup complete")
        return True

    def update(self):
        """Rotates the robot while battery is low"""
        # Read battery warning from blackboard
        battery_low = self.blackboard.get("battery_low_warning")

        if battery_low:
            # Battery is low - rotate for recharge
            twist = Twist()
            twist.angular.z = self.ang_vel
            self.cmd_vel_pub.publish(twist)
            return pt.common.Status.RUNNING  # Keep this behavior active
        else:
            # Battery is OK - this behavior not needed
            return pt.common.Status.FAILURE  # Let selector try next behavior

    def terminate(self, new_status):
        """Stop rotation when behavior terminates"""
        # Only stop if we were actually running
        if new_status == pt.common.Status.INVALID:
            twist = Twist()
            twist.angular.z = 0.0
            self.cmd_vel_pub.publish(twist)

        return super().terminate(new_status)


class StopMotion(pt.behaviour.Behaviour):
    """Stops the robot when collision is detected"""

    def __init__(self, name="StopMotion", topic_name="/cmd_vel"):
        super(StopMotion, self).__init__(name)
        self.topic_name = topic_name
        self.cmd_vel_pub = None
        self.blackboard = None

    def setup(self, **kwargs):
        """Setup publisher and blackboard client"""
        try:
            self.node = kwargs["node"]
        except KeyError as e:
            error_message = f"didn't find 'node' in setup's kwargs [{self.qualified_name}]"
            raise KeyError(error_message) from e

        # Create publisher
        self.cmd_vel_pub = self.node.create_publisher(Twist, self.topic_name, 10)

        # Attach to blackboard
        self.blackboard = self.attach_blackboard_client(name=self.name)
        self.blackboard.register_key(key="collision_warning", access=pt.common.Access.READ)

        self.node.get_logger().info(f"[{self.name}] ✅ Setup complete")
        return True

    def update(self):
        """Stops the robot if collision warning is active"""
        collision_warning = self.blackboard.get("collision_warning")

        if collision_warning:
            # Collision detected - stop the robot
            twist = Twist()
            twist.linear.x = 0.0
            twist.angular.z = 0.0
            self.cmd_vel_pub.publish(twist)
            return pt.common.Status.RUNNING  # Keep this behavior active
        else:
            # No collision - this behavior not needed
            return pt.common.Status.FAILURE  # Let selector try next behavior

    def terminate(self, new_status):
        """Cleanup when behavior terminates"""
        return super().terminate(new_status)


class BatteryStatus2bb(ptr.subscribers.ToBlackboard):
    """Subscribes to battery level and updates blackboard with low battery warning"""

    def __init__(self, battery_topic_name: str = "/battery_level", name: str = "Battery2BB", threshold_low: float = 20.0, threshold_ok: float = 85.0):
        super().__init__(
            name=name,
            topic_name=battery_topic_name,
            topic_type=Float32,
            blackboard_variables={"battery": "data"},
            initialise_variables={"battery": 100.0},
            clearing_policy=pt.common.ClearingPolicy.NEVER,
            qos_profile=ptr.utilities.qos_profile_unlatched(),
        )

        self.threshold_low = threshold_low
        self.threshold_ok = threshold_ok
        self.last_logged_state = None  # Track state to reduce logging

        # Register blackboard keys for writing
        self.blackboard.register_key(key="battery_low_warning", access=pt.common.Access.WRITE)

        # Initialize warning flag
        self.blackboard.battery_low_warning = False

    def update(self):
        """Check battery level and update warning flag with hysteresis"""
        # First call parent to update blackboard with raw battery data
        status = super().update()

        # Check if battery data exists on blackboard yet
        try:
            battery_level = self.blackboard.get("battery")
        except KeyError:
            return pt.common.Status.SUCCESS

        # Hysteresis logic to prevent oscillation
        new_warning = self.blackboard.battery_low_warning

        if battery_level < self.threshold_low:
            # Battery low - set warning
            new_warning = True
        elif battery_level > self.threshold_ok:
            # Battery recharged - clear warning
            new_warning = False
        # else: maintain current state (hysteresis)

        # Only log on state changes
        if new_warning != self.last_logged_state:
            if new_warning:
                self.logger.info(f"[{self.name}] 🔋 Battery LOW: {battery_level:.1f}%")
            else:
                self.logger.info(f"[{self.name}] ✅ Battery OK: {battery_level:.1f}%")
            self.last_logged_state = new_warning

        self.blackboard.battery_low_warning = new_warning
        return pt.common.Status.SUCCESS


class LaserScan2bb(ptr.subscribers.ToBlackboard):
    """Subscribes to laser scan and updates blackboard with collision warning"""

    def __init__(self, topic_name: str = "/scan", name: str = "Scan2BB", collision_distance: float = 0.5, safe_distance: float = 0.75):
        super().__init__(
            name=name,
            topic_name=topic_name,
            topic_type=LaserScan,
            blackboard_variables={"laser_scan": "ranges"},
            clearing_policy=pt.common.ClearingPolicy.NEVER,
            qos_profile=QoSProfile(reliability=QoSReliabilityPolicy.BEST_EFFORT, history=QoSHistoryPolicy.KEEP_LAST, depth=10),
        )

        self.collision_distance = collision_distance
        self.safe_distance = safe_distance
        self.last_logged_state = None  # Track state to reduce logging

        # Register blackboard key for collision warning
        self.blackboard.register_key(key="collision_warning", access=pt.common.Access.WRITE)

        # Initialize warning flag
        self.blackboard.collision_warning = False

    def update(self):
        """Check laser scan for obstacles and update collision warning with hysteresis"""
        # Call parent to update blackboard with raw scan data
        status = super().update()

        # Check if laser_scan data exists on blackboard yet
        try:
            ranges = self.blackboard.get("laser_scan")
        except KeyError:
            return pt.common.Status.SUCCESS

        if ranges:
            # Find minimum valid distance
            valid_ranges = [r for r in ranges if 0.1 < r < 100.0]

            if valid_ranges:
                min_distance = min(valid_ranges)
                new_warning = self.blackboard.collision_warning

                # Hysteresis logic
                if min_distance < self.collision_distance:
                    # Obstacle too close - set warning
                    new_warning = True
                elif min_distance > self.safe_distance:
                    # Safe distance - clear warning
                    new_warning = False
                # else: maintain current state (hysteresis)

                # Only log on state changes
                if new_warning != self.last_logged_state:
                    if new_warning:
                        self.logger.info(f"[{self.name}] ⚠️  COLLISION: {min_distance:.2f}m")
                    else:
                        self.logger.info(f"[{self.name}] ✅ Safe: {min_distance:.2f}m")
                    self.last_logged_state = new_warning

                self.blackboard.collision_warning = new_warning

        return pt.common.Status.SUCCESS


ModuleNotFoundError: No module named 'py_trees'

Now, set up and initialise your behaviour tree in the cell below. [10 points]

In [4]:
"""
Behavior Tree Structure for Robot Safety System
"""

import py_trees as pt

from robile_safety_bt.behaviours import BatteryStatus2bb, LaserScan2bb, Rotate, StopMotion


def create_root(config) -> pt.behaviour.Behaviour:
    """
    Creates behavior tree for robot safety system.

    Tree Structure:
        Root (Parallel)
        ├── Topics2BB (Sequence) - Update blackboard with sensor data
        │   ├── BatteryStatus2bb
        │   └── LaserScan2bb
        └── Priorities (Selector) - Execute behaviors based on priorities
            ├── StopMotion (highest priority - safety first!)
            ├── Rotate (medium priority - battery management)
            └── Idle (lowest priority - default behavior)

    Args:
        config: SafetyConfig object with all thresholds

    Returns:
        Root node of the behavior tree
    """

    # Root - Parallel node runs both branches simultaneously
    root = pt.composites.Parallel(name="Root", policy=pt.common.ParallelPolicy.SuccessOnAll(synchronise=False))

    # Branch 1: Topics2BB - Update blackboard with sensor data
    topics2BB = pt.composites.Sequence(name="Topics2BB", memory=False)

    # Branch 2: Priorities - Execute behaviors based on conditions
    priorities = pt.composites.Selector(name="Priorities", memory=False)

    # Behaviors for Topics2BB
    battery2bb = BatteryStatus2bb(name="Battery2BB", battery_topic_name="/battery_level", threshold_low=config.low_battery_threshold, threshold_ok=config.battery_ok_threshold)

    scan2bb = LaserScan2bb(name="Scan2BB", topic_name="/scan", collision_distance=config.collision_distance, safe_distance=config.safe_distance)

    # Behaviors for Priorities
    stop_motion = StopMotion(name="StopMotion", topic_name="/cmd_vel")

    rotate = Rotate(name="Rotate", topic_name="/cmd_vel", ang_vel=config.rotation_speed)

    idle = pt.behaviours.Running(name="Idle")

    # Build tree structure
    root.add_children([topics2BB, priorities])
    topics2BB.add_children([battery2bb, scan2bb])
    priorities.add_children([stop_motion, rotate, idle])

    return root


"""
Robot Safety using Behavior Trees (py_trees)
Entry point for the behavior tree safety system
"""

import sys
from dataclasses import dataclass

import py_trees.console as console
import py_trees_ros as ptr
import rclpy
from rclpy.node import Node

from robile_safety_bt.tree import create_root


@dataclass
class SafetyConfig:
    """Centralized configuration - Single source of truth!"""

    # Battery thresholds
    low_battery_threshold: float = 20.0
    battery_ok_threshold: float = 85.0

    # Collision thresholds
    collision_distance: float = 0.5
    safe_distance: float = 0.75

    # Robot motion
    rotation_speed: float = 5.0
    monitor_rate: float = 10.0

    @classmethod
    def from_node(cls, node: Node):
        """Load configuration from ROS2 parameters"""
        node.declare_parameter("low_battery_threshold", cls.low_battery_threshold)
        node.declare_parameter("battery_ok_threshold", cls.battery_ok_threshold)
        node.declare_parameter("collision_distance", cls.collision_distance)
        node.declare_parameter("safe_distance", cls.safe_distance)
        node.declare_parameter("rotation_speed", cls.rotation_speed)
        node.declare_parameter("monitor_rate", cls.monitor_rate)

        return cls(
            low_battery_threshold=node.get_parameter("low_battery_threshold").value,
            battery_ok_threshold=node.get_parameter("battery_ok_threshold").value,
            collision_distance=node.get_parameter("collision_distance").value,
            safe_distance=node.get_parameter("safe_distance").value,
            rotation_speed=node.get_parameter("rotation_speed").value,
            monitor_rate=node.get_parameter("monitor_rate").value,
        )

    def log_config(self, logger):
        """Pretty print configuration"""
        logger.info("=" * 60)
        logger.info("🌳 Behavior Tree Safety Configuration:")
        logger.info(f"  Battery Low:      {self.low_battery_threshold:.1f}%")
        logger.info(f"  Battery OK:       {self.battery_ok_threshold:.1f}%")
        logger.info(f"  Collision Dist:   {self.collision_distance:.2f}m")
        logger.info(f"  Safe Dist:        {self.safe_distance:.2f}m")
        logger.info(f"  Rotation Speed:   {self.rotation_speed:.1f} rad/s")
        logger.info("=" * 60)


def main():
    """Initializes and executes the behavior tree"""

    rclpy.init(args=None)

    # Create a temporary node to load config
    temp_node = Node("safety_bt_config_loader")
    config = SafetyConfig.from_node(temp_node)
    config.log_config(temp_node.get_logger())
    temp_node.destroy_node()

    # Create behavior tree root
    root = create_root(config)

    # Create behavior tree with py_trees_ros
    tree = ptr.trees.BehaviourTree(root=root, unicode_tree_debug=True)

    try:
        tree.setup(timeout=30.0)
        tree.node.get_logger().info("✅ Behavior tree setup complete")
        tree.node.get_logger().info("🚀 Starting execution...")
    except ptr.exceptions.TimedOutError as e:
        console.logerror(console.red + f"❌ Failed to setup tree: {str(e)}" + console.reset)
        tree.shutdown()
        rclpy.try_shutdown()
        sys.exit(1)
    except KeyboardInterrupt:
        console.logerror("⏹️  Tree setup interrupted")
        tree.shutdown()
        rclpy.try_shutdown()
        sys.exit(1)

    # Tick frequency (100ms = 10 Hz)
    tick_period_ms = int(1000 / config.monitor_rate)
    tree.tick_tock(period_ms=tick_period_ms)

    try:
        rclpy.spin(tree.node)
    except (KeyboardInterrupt, rclpy.executors.ExternalShutdownException):
        pass
    finally:
        tree.node.get_logger().info("⏹️  Shutting down behavior tree")
        tree.shutdown()
        rclpy.try_shutdown()


if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'py_trees'

## Setting up Your System for Testing

1. (If not done already) Please set up Ubuntu 22.04, ROS2 humble, and the Robile simulation by following the [documentation](https://robile-amr.readthedocs.io/en/humble/).

2. Clone the `executive_smach` repository in the src folder of your workspace, and also install the `py-trees-ros` package:
```
cd ~/ros2_ws/src/  
git clone -b ros2 https://github.com/ros/executive_smach.git  
sudo apt-get install ros-humble-py-trees ros-humble-py-trees-ros ros-humble-py-trees-ros-interfaces xcb
```

From the workspace directory, build the workspace:
```
cd ~/ros2_ws/
colcon build --symlink-install
```

Now source the workspace setup file:
```
source install/setup.bash
```
3. Create a new ROS2 python package and integrate your implementation in it

## Testing Instructions

Use the following steps to test your implementation:
- Run the Robile in simulation
- After integrating your implementation in your local ROS workspace, launch your components to test the functionalities. **Note that you need to test the state machine and behaviour tree implementations independently to verify that the robot behaves exactly the same in both cases.**

**In your submission, please include screenshots to verify your implementation, and link them in the cell further below.**

As already mentioned before, as the battery percentage is not readily available in simulation, please publish the battery percentage values manually. For instance, if the topic `/battery_voltage` is used for monitoring the battery status, you should be able to publish a battery percentage value of your choice to verify your implementation, e.g.:
```  
ros2 topic  pub /battery_voltage std_msgs/msg/Float32 "data: 50.0"
```

Finally, behaviour tree visualization is not released on ROS2 humble yet, but the changes in the behaviour tree can be monitored by running the following command, which is helpful for debugging:
```
py-trees-tree-watcher
```

The following is a sample visualisation when the robot is about to collide:

![collision avoidance BT](figures/BT_watcher.png)

For getting a better intuition of behaviour trees, another sample visualisation of a similar task, but with a slightly different structure can be found below:

![collison and battery low](figures/collision_battery.png)

**Discuss any observations from your tests and include screenshots that verify your implementation in the cell below. [10 points]**

Github -> https://github.com/emircetinmemis/ser1

Demo -> https://www.youtube.com/watch?v=12EEkJe1wfI

Please take a look at full implementations through github, and watch my demo.

Observations:

Both implementations work correctly for the safety requirements, but they handle things differently. SMACH uses a state machine where the robot is always in one state at a time - monitoring, rotating, or stopped. This makes it easy to understand what's happening because you can see exactly which state is active. However, when the robot is rotating due to low battery, it has to wait until the next monitoring cycle to detect a collision. Behavior Trees run differently because they check sensors constantly in parallel while deciding which action to take. This means the collision detection reacts faster, usually within 100ms compared to 200-300ms for SMACH.

From a development perspective, SMACH was more straightforward to implement initially since states and transitions are explicit in the code. Behavior Trees took more effort to understand, especially the blackboard concept for sharing data between behaviors. However, when we needed to adjust priorities or add new behaviors, the BT structure was easier to modify - we just reordered nodes in the tree rather than changing multiple transition definitions. For this safety system specifically, both work fine, but the BT version feels more responsive when testing collision avoidance while the battery is low. SMACH uses less CPU though, which could matter on resource-constrained robots.
